# Mosaico 2021

Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Mosaicos Patagonia

2021-08-05 13:07:09 

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [160]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Colors Bars

In [4]:
import geemap.colormaps as cm

In [5]:
# cm.palettes.terrain

In [6]:
color_ndvi = cm.palettes.ndvi

In [7]:
color_ndwi = cm.palettes.ndwi

In [8]:
color_terrain = cm.palettes.terrain

In [9]:
color_glaciar = cm.palettes.PuBuGn_r

In [10]:
color_nubes = cm.palettes.inferno

In [11]:
# cm.list_colormaps()

In [12]:
# cm.plot_colormaps(width=12, height=0.4)

## Funciones

In [13]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [14]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [15]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [16]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [17]:
def addSAVI(image):
    savi = image.expression(
        '((b5 - b4)/(b5 + b4 + 0.5))*1.5', {
            'b4':image.select('B4'),
            'b5':image.select('B5')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

In [18]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B3','B6']).rename('NDSI')
    return image.addBands(ndsi)

In [19]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMM'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [20]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [21]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [22]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [23]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 2021

In [24]:
start_date = '2020-10-01'
end_date = '2021-03-31'

l8_2021 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [25]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [26]:
vis_nieve = {
  'bands': ['B6', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [27]:
coleccion2021 = l8_2021.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [28]:
QM_NDVI_2021 = coleccion2021.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2021 = coleccion2021.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2021 = coleccion2021.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2021 = coleccion2021.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2021 = coleccion2021.qualityMosaic('NDSI') # Normalized difference snow index

In [29]:
Map.addLayer(QM_NDVI_2021, vizParams, 'QM de NDVI 2021')
Map.addLayer(QM_NDWI_2021, vizParams, 'QM de NDWI 2021')
Map.addLayer(QM_SAVI_2021, vizParams, 'QM de SAVI 2021')
Map.addLayer(QM_NDGI_2021, vis_nieve, 'QM de NDGI 2021')
Map.addLayer(QM_NDSI_2021, vis_nieve, 'QM de NDSI 2021')
# Map 

In [30]:
c2021_sn = coleccion2021.map(Cloud20)
NDVI_2021_sn = c2021_sn.qualityMosaic('NDVI')
NDWI_2021_sn = c2021_sn.qualityMosaic('NDWI')
SAVI_2021_sn = c2021_sn.qualityMosaic('SAVI')
NDGI_2021_sn = c2021_sn.qualityMosaic('NDGI')
NDSI_2021_sn = c2021_sn.qualityMosaic('NDSI')

In [31]:
Map2.addLayer(NDVI_2021_sn, vizParams, 'QMsn de NDVI 2021')
Map2.addLayer(NDWI_2021_sn, vizParams, 'QMsn de NDWI 2021')
Map2.addLayer(SAVI_2021_sn, vizParams, 'QMsn de SAVI 2021')
Map2.addLayer(NDGI_2021_sn, vis_nieve, 'QMsn de NDGI 2021')
Map2.addLayer(NDSI_2021_sn, vis_nieve, 'QMsn de NDSI 2021')
# Map2

### 2020

In [32]:
start_date = '2019-10-01'
end_date = '2020-03-31'

l8_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [33]:
coleccion2020 = l8_2020.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [34]:
QM_NDVI_2020 = coleccion2020.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2020 = coleccion2020.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2020 = coleccion2020.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2020 = coleccion2020.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2020 = coleccion2020.qualityMosaic('NDSI') # Normalized difference snow index

In [35]:
Map.addLayer(QM_NDVI_2020, vizParams, 'QM de NDVI 2020')
Map.addLayer(QM_NDWI_2020, vizParams, 'QM de NDWI 2020')
Map.addLayer(QM_SAVI_2020, vizParams, 'QM de SAVI 2020')
Map.addLayer(QM_NDGI_2020, vis_nieve, 'QM de NDGI 2020')
Map.addLayer(QM_NDSI_2020, vis_nieve, 'QM de NDSI 2020')
# Map

In [36]:
c2020_sn = coleccion2020.map(Cloud20)
NDVI_2020_sn = c2020_sn.qualityMosaic('NDVI')
NDWI_2020_sn = c2020_sn.qualityMosaic('NDWI')
SAVI_2020_sn = c2020_sn.qualityMosaic('SAVI')
NDGI_2020_sn = c2020_sn.qualityMosaic('NDGI')
NDSI_2020_sn = c2020_sn.qualityMosaic('NDSI')

In [37]:
Map2.addLayer(NDVI_2020_sn, vizParams, 'QMsn de NDVI 2020')
Map2.addLayer(NDWI_2020_sn, vizParams, 'QMsn de NDWI 2020')
Map2.addLayer(SAVI_2020_sn, vizParams, 'QMsn de SAVI 2020')
Map2.addLayer(NDGI_2020_sn, vis_nieve, 'QMsn de NDGI 2020')
Map2.addLayer(NDSI_2020_sn, vis_nieve, 'QMsn de NDSI 2020')
# Map2

### 2019

In [38]:
start_date = '2018-10-01'
end_date = '2019-03-31'

l8_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [39]:
coleccion2019 = l8_2019.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [40]:
QM_NDVI_2019 = coleccion2019.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2019 = coleccion2019.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2019 = coleccion2019.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2019 = coleccion2019.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2019 = coleccion2019.qualityMosaic('NDSI') # Normalized difference snow index

In [41]:
Map3.addLayer(QM_NDVI_2019, vizParams, 'QM de NDVI 2019')
Map3.addLayer(QM_NDWI_2019, vizParams, 'QM de NDWI 2019')
Map3.addLayer(QM_SAVI_2019, vizParams, 'QM de SAVI 2019')
Map3.addLayer(QM_NDGI_2019, vis_nieve, 'QM de NDGI 2019')
Map3.addLayer(QM_NDSI_2019, vis_nieve, 'QM de NDSI 2019')
# Map3

In [42]:
c2019_sn = coleccion2019.map(Cloud20)
NDVI_2019_sn = c2019_sn.qualityMosaic('NDVI')
NDWI_2019_sn = c2019_sn.qualityMosaic('NDWI')
SAVI_2019_sn = c2019_sn.qualityMosaic('SAVI')
NDGI_2019_sn = c2019_sn.qualityMosaic('NDGI')
NDSI_2019_sn = c2019_sn.qualityMosaic('NDSI')

In [43]:
Map4.addLayer(NDVI_2019_sn, vizParams, 'QMsn de NDVI 2019')
Map4.addLayer(NDWI_2019_sn, vizParams, 'QMsn de NDWI 2019')
Map4.addLayer(SAVI_2019_sn, vizParams, 'QMsn de SAVI 2019')
Map4.addLayer(NDGI_2019_sn, vis_nieve, 'QMsn de NDGI 2019')
Map4.addLayer(NDSI_2019_sn, vis_nieve, 'QMsn de NDSI 2019')
# Map4

### 2018

In [44]:
start_date = '2017-10-01'
end_date = '2018-03-31'

l8_2018 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [45]:
coleccion2018 = l8_2018.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [46]:
QM_NDVI_2018 = coleccion2018.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2018 = coleccion2018.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2018 = coleccion2018.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2018 = coleccion2018.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2018 = coleccion2018.qualityMosaic('NDSI') # Normalized difference snow index

In [47]:
Map3.addLayer(QM_NDVI_2018, vizParams, 'QM de NDVI 2018')
Map3.addLayer(QM_NDWI_2018, vizParams, 'QM de NDWI 2018')
Map3.addLayer(QM_SAVI_2018, vizParams, 'QM de SAVI 2018')
Map3.addLayer(QM_NDGI_2018, vis_nieve, 'QM de NDGI 2018')
Map3.addLayer(QM_NDSI_2018, vis_nieve, 'QM de NDSI 2018')
# Map3

In [48]:
c2018_sn = coleccion2018.map(Cloud20)
NDVI_2018_sn = c2018_sn.qualityMosaic('NDVI')
NDWI_2018_sn = c2018_sn.qualityMosaic('NDWI')
SAVI_2018_sn = c2018_sn.qualityMosaic('SAVI')
NDGI_2018_sn = c2018_sn.qualityMosaic('NDGI')
NDSI_2018_sn = c2018_sn.qualityMosaic('NDSI')

In [49]:
Map4.addLayer(NDVI_2018_sn, vizParams, 'QMsn de NDVI 2018')
Map4.addLayer(NDWI_2018_sn, vizParams, 'QMsn de NDWI 2018')
Map4.addLayer(SAVI_2018_sn, vizParams, 'QMsn de SAVI 2018')
Map4.addLayer(NDGI_2018_sn, vis_nieve, 'QMsn de NDGI 2018')
Map4.addLayer(NDSI_2018_sn, vis_nieve, 'QMsn de NDSI 2018')
# Map4

### 2017

In [50]:
start_date = '2016-10-01'
end_date = '2017-03-31'

l8_2017 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [51]:
coleccion2017 = l8_2017.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [52]:
QM_NDVI_2017 = coleccion2017.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2017 = coleccion2017.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2017 = coleccion2017.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2017 = coleccion2017.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2017 = coleccion2017.qualityMosaic('NDSI') # Normalized difference snow index

In [53]:
Map7.addLayer(QM_NDVI_2017, vizParams, 'QM de NDVI 2017')
Map7.addLayer(QM_NDWI_2017, vizParams, 'QM de NDWI 2017')
Map7.addLayer(QM_SAVI_2017, vizParams, 'QM de SAVI 2017')
Map7.addLayer(QM_NDGI_2017, vis_nieve, 'QM de NDGI 2017')
Map7.addLayer(QM_NDSI_2017, vis_nieve, 'QM de NDSI 2017')
# Map7

In [54]:
c2017_sn = coleccion2017.map(Cloud20)
NDVI_2017_sn = c2017_sn.qualityMosaic('NDVI')
NDWI_2017_sn = c2017_sn.qualityMosaic('NDWI')
SAVI_2017_sn = c2017_sn.qualityMosaic('SAVI')
NDGI_2017_sn = c2017_sn.qualityMosaic('NDGI')
NDSI_2017_sn = c2017_sn.qualityMosaic('NDSI')

In [55]:
Map8.addLayer(NDVI_2017_sn, vizParams, 'QMsn de NDVI 2017')
Map8.addLayer(NDWI_2017_sn, vizParams, 'QMsn de NDWI 2017')
Map8.addLayer(SAVI_2017_sn, vizParams, 'QMsn de SAVI 2017')
Map8.addLayer(NDGI_2017_sn, vis_nieve, 'QMsn de NDGI 2017')
Map8.addLayer(NDSI_2017_sn, vis_nieve, 'QMsn de NDSI 2017')
# Map8

#### Colección agua 2019-2021

In [152]:
start_date = '2018-10-01'
end_date = '2021-03-31'

l8_2021_sur = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [153]:
coleccion2021_sur = l8_2021_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [154]:
QM_NDVI_2021_sur = coleccion2021_sur.qualityMosaic('NDVI') # Normalized difference vegetation Index
QM_NDWI_2021_sur = coleccion2021_sur.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2021_sur = coleccion2021_sur.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2021_sur = coleccion2021_sur.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2021_sur = coleccion2021_sur.qualityMosaic('NDSI') # Normalized difference snow index

In [155]:
c2021_sur_sn = coleccion2021_sur.map(Cloud20)
NDVI_2021_sur_sn = c2021_sur_sn.qualityMosaic('NDVI')
NDWI_2021_sur_sn = c2021_sur_sn.qualityMosaic('NDWI')
SAVI_2021_sur_sn = c2021_sur_sn.qualityMosaic('SAVI')
NDGI_2021_sur_sn = c2021_sur_sn.qualityMosaic('NDGI')
NDSI_2021_sur_sn = c2021_sur_sn.qualityMosaic('NDSI')

## Indices por colección

### Índices 2021

In [56]:
NDVI_2021 = NDVI_2021_sn.select('NDVI')
NDWI_2021 = NDWI_2021_sn.select('NDWI')
SAVI_2021 = SAVI_2021_sn.select('SAVI')
NDGI_2021 = NDGI_2021_sn.select('NDGI')
NDSI_2021 = NDSI_2021_sn.select('NDSI')

In [137]:
Map9.addLayer(NDVI_2021, {'palette': color_terrain}, 'QMsn de NDVI 2021')
Map9.addLayer(NDWI_2021, {'palette': color_ndwi}, 'QMsn de NDWI 2021')
Map9.addLayer(SAVI_2021, {'palette': color_ndvi}, 'QMsn de SAVI 2021')
Map9.addLayer(NDGI_2021, {'palette': color_glaciar}, 'QMsn de NDGI 2021')
Map9.addLayer(NDSI_2021, {'palette': color_ndwi}, 'QMsn de NDSI 2021')
Map9

Map(bottom=174548.0, center=[-50.882243111139694, -72.16441067912926], controls=(WidgetControl(options=['posit…

### Índices 2020

In [58]:
NDVI_2020 = NDVI_2020_sn.select('NDVI')
NDWI_2020 = NDWI_2020_sn.select('NDWI')
SAVI_2020 = SAVI_2020_sn.select('SAVI')
NDGI_2020 = NDGI_2020_sn.select('NDGI')
NDSI_2020 = NDSI_2020_sn.select('NDSI')

In [59]:
Map5.addLayer(NDVI_2020, {'palette': color_terrain}, 'QMsn de NDVI 2020')
Map5.addLayer(NDWI_2020, {'palette': color_ndwi}, 'QMsn de NDWI 2020')
Map5.addLayer(SAVI_2020, {'palette': color_ndvi}, 'QMsn de SAVI 2020')
# Map5.addLayer(NDGI_2020, {'palette': color_glaciar}, 'QMsn de NDGI 2020')
# Map5.addLayer(NDSI_2020, {'palette': color_ndwi}, 'QMsn de NDSI 2020')
Map5

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 2019

In [60]:
NDVI_2019 = NDVI_2019_sn.select('NDVI')
NDWI_2019 = NDWI_2019_sn.select('NDWI')
SAVI_2019 = SAVI_2019_sn.select('SAVI')
NDGI_2019 = NDGI_2019_sn.select('NDGI')
NDSI_2019 = NDSI_2019_sn.select('NDSI')

In [61]:
# Map6.addLayer(NDVI_2019, {'palette': color_terrain}, 'QMsn de NDVI 2019')
# Map6.addLayer(NDWI_2019, {'palette': color_ndwi}, 'QMsn de NDWI 2019')
# Map6.addLayer(SAVI_2019, {'palette': color_ndvi}, 'QMsn de SAVI 2019')
# Map6.addLayer(NDGI_2019, {'palette': color_glaciar}, 'QMsn de NDGI 2019')
# Map6.addLayer(NDSI_2019, {'palette': color_ndwi}, 'QMsn de NDSI 2019')
# Map6

### Índices 2018

In [62]:
NDVI_2018 = NDVI_2018_sn.select('NDVI')
NDWI_2018 = NDWI_2018_sn.select('NDWI')
SAVI_2018 = SAVI_2018_sn.select('SAVI')
NDGI_2018 = NDGI_2018_sn.select('NDGI')
NDSI_2018 = NDSI_2018_sn.select('NDSI')

In [63]:
# Map6.addLayer(NDVI_2018, {'palette': color_terrain}, 'QMsn de NDVI 2018')
# Map6.addLayer(NDWI_2018, {'palette': color_ndwi}, 'QMsn de NDWI 2018')
# Map6.addLayer(SAVI_2018, {'palette': color_ndvi}, 'QMsn de SAVI 2018')
# Map6.addLayer(NDGI_2018, {'palette': color_glaciar}, 'QMsn de NDGI 2018')
# Map6.addLayer(NDSI_2018, {'palette': color_ndwi}, 'QMsn de NDSI 2018')
# Map6

## Generación de Mosaico

In [64]:
NDVI_2021_landsat = QM_NDVI_2021.set('SENSOR_ID', 'OLI_TIRS')

In [65]:
cloud_2021 = ee.Algorithms.Landsat.simpleCloudScore(NDVI_2021_landsat)

In [66]:
cloud_2021 = cloud_2021.select('cloud')

In [67]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [68]:
resultados= [cloud_2021, QM_NDVI_2021]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes 2021',
    'QM NDVI 2021']

In [69]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [70]:
def llenado(image):
    return image.where(cloud_2021.gt(20),NDVI_2021_sn)

In [71]:
img1 = llenado(QM_NDVI_2021)

In [72]:
img1_landsat = img1.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [73]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [74]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [75]:
def llenado2(image):
    return image.where(cloud_img1.gt(20), NDVI_2020_sn)

In [76]:
img2 = llenado2(img1)

In [77]:
img2_landsat = img2.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [78]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [79]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [80]:
def llenado3(image):
    return image.where(cloud_img2.gt(20), NDVI_2019_sn)

In [81]:
img3 = llenado3(img2)

In [82]:
img3_landsat = img3.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [83]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [84]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [85]:
def llenado4(image):
    return image.where(cloud_img3.gt(20), NDVI_2018_sn)

In [86]:
img4 = llenado4(img3)

In [87]:
img4_landsat = img4.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [88]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [89]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [90]:
def llenado5(image):
    return image.where(cloud_img4.gt(20), NDVI_2017_sn)

In [91]:
img5 = llenado5(img4)

In [92]:
img5_landsat = img5.set('SENSOR_ID', 'OLI_TIRS')
cloud_img5 = ee.Algorithms.Landsat.simpleCloudScore(img5_landsat)
cloud_img5 = cloud_img5.select('cloud')

In [93]:
resultados= [cloud_img5, img5]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 5',
    'Mosaico 5']

In [94]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [156]:
def llenado6(image):
    return image.where(NDVI_2021.lt(0.25).And(NDWI_2021.gt(0.3)), NDWI_2021_sur_sn)

In [157]:
img6 = llenado6(img5)

In [158]:
resultados= [img5, img6]
labels = [
    'Mosaico 5',
    'Mosaico 6']
vis_params3=  [
     vizParams, 
    vizParams]

In [159]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params3, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [161]:
Map10.addLayer(img5, vizParams, 'Mosaico 5')
Map10.addLayer(img6, vizParams, 'Mosaico 6')
Map10

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…